In [23]:
import rasterio
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box
import pylandstats as pls
from rasterio.mask import mask
from rasterio.features import geometry_mask
import warnings
warnings.filterwarnings('ignore')


Set file paths

In [2]:
Impervious = r"C:\Users\alekb\Downloads\Annual_NLCD_ImpDsc_2016_CU_C1V1.tif"
Flood_Points = r"C:\Users\alekb\Downloads\shp\filtered_flooding_points_over_one_day_nopr.shp"
#Impervious = "/anvil/projects/x-cis250634/team5/data_0718/Annual_NLCD_ImpDsc_2016_CU_C1V1.tif"
#Flood_Points = "/anvil/projects/x-cis250634/team5/shp/filtered_flooding_points_over_one_day_nopr.shp"

Import and pre-process NLCD impervious layer

In [ ]:
# Import NLCD Data
print("Loading impervious layer...")
with rasterio.open(Impervious) as src:
    impervious_data = src.read(1)
    impervious_meta = src.meta
    print(f"Raster shape: {impervious_data.shape}")
    print(f"CRS: {src.crs}")
    print(f"Transform: {src.transform}")

# Recode the data: 1->1, 2->2, everything else->0
recoded_data = np.where((impervious_data == 1) | (impervious_data == 2), impervious_data, 0)

Loading impervious layer...


Loading Flood Points

In [5]:
# Import flood points
print("Loading flood points...")
flood_points = gpd.read_file(Flood_Points)
print(f"Number of flood points: {len(flood_points)}")
print(f"CRS: {flood_points.crs}")
print(f"Columns: {flood_points.columns.tolist()}")

Loading flood points...
Number of flood points: 1219
CRS: EPSG:4326
Columns: ['latitude_d', 'longitude_', 'latitude', 'longitude', 'eventName', 'hwmTypeNam', 'hwmQuality', 'verticalDa', 'verticalMe', 'approvalMe', 'markerName', 'horizontal', 'horizont_1', 'flagMember', 'surveyMemb', 'site_no', 'siteDescri', 'sitePriori', 'networkNam', 'stateName', 'countyName', 'siteZone', 'sitePermHo', 'site_latit', 'site_longi', 'hwm_id', 'waterbody', 'site_id', 'event_id', 'hwm_type_i', 'hwm_qualit', 'hwm_locati', 'survey_dat', 'elev_ft', 'vdatum_id', 'vcollect_m', 'bank', 'approval_i', 'marker_id', 'height_abo', 'hcollect_m', 'peak_summa', 'hwm_notes', 'hwm_enviro', 'flag_date', 'stillwater', 'hdatum_id', 'flag_membe', 'survey_mem', 'uncertaint', 'hwm_uncert', 'hwm_label', 'last_updat', 'last_upd_1', 'horizont_2', 'vertical_c', 'horizont_3', 'hwm_types', 'hwm_qual_1', 'event', 'peak_sum_1', 'survey_m_1', 'marker', 'approval', 'files', 'site', 'vertical_d', 'flag_mem_1', 'Links', 'p_latitude', 'p_lo

Create 1.5 km square buffers around flood points

In [7]:
# Create 1.5 km square buffers around each flood point

# First, check if we need to reproject to a projected CRS for accurate distance calculations
print(f"Original CRS: {flood_points.crs}")

# If the CRS is geographic (lat/lon), reproject to a suitable projected CRS
if flood_points.crs.is_geographic:
    # Use UTM zone (this assumes data is in US - adjust if needed)
    flood_points_proj = flood_points.to_crs('EPSG:3857')  # Web Mercator for general use
    print(f"Reprojected to: {flood_points_proj.crs}")
else:
    flood_points_proj = flood_points.copy()

# Create square buffers (1.5 km = 1500 meters)
buffer_distance = 1500  # meters

def create_square_buffer(point, distance):
    """Create a square buffer around a point"""
    x, y = point.x, point.y
    return box(x - distance, y - distance, x + distance, y + distance)

# Apply square buffer to each point
print("Creating square buffers...")
flood_points_proj['square_buffer'] = flood_points_proj.geometry.apply(
    lambda point: create_square_buffer(point, buffer_distance)
)

# Create a new GeoDataFrame with the buffers as geometry
buffered_points = flood_points_proj.copy()
buffered_points.geometry = buffered_points['square_buffer']

# Reproject back to original CRS if needed
if flood_points.crs.is_geographic:
    buffered_points = buffered_points.to_crs(flood_points.crs)

print(f"Created {len(buffered_points)} square buffers")
print(f"Buffer area: {(buffer_distance * 2) ** 2 / 1000000:.2f} km²")

Original CRS: EPSG:4326
Reprojected to: EPSG:3857
Creating square buffers...
Created 1219 square buffers
Buffer area: 9.00 km²


Calculate landscape metrics for impervious surfaces within buffers

In [27]:
# Complete landscape metrics calculation with correct ID column
import pylandstats as pls
from rasterio.mask import mask
import warnings
warnings.filterwarnings('ignore')

# Initialize lists to store results
buffer_metrics = []

print("Calculating landscape metrics for each buffer...")

# Open the raster file to get the transform and CRS
with rasterio.open(Impervious) as src:
    raster_crs = src.crs
    raster_transform = src.transform
    
    # Ensure buffers are in the same CRS as the raster
    if buffered_points.crs != raster_crs:
        buffered_points_raster_crs = buffered_points.to_crs(raster_crs)
    else:
        buffered_points_raster_crs = buffered_points.copy()
    
    for idx, buffer_geom in enumerate(buffered_points_raster_crs.geometry):
        try:
            # Mask the raster data with the buffer geometry
            masked_data, masked_transform = mask(src, [buffer_geom], crop=True, filled=False)
            masked_array = masked_data[0]  # Get the first band
            
            # Replace masked values with a no-data value for pylandstats
            masked_array = np.where(masked_array.mask, -1, masked_array.data)
            
            # Apply our recoding to the masked data
            recoded_masked = np.where((masked_array == 1) | (masked_array == 2), masked_array, 0)
            
            # Create Landscape object
            ls = pls.Landscape(recoded_masked, res=(30, 30))  # Assuming 30m resolution
            
            # Get the original ID from shapefile
            original_id = flood_points.iloc[idx]['ID']
            
            # Calculate metrics
            metrics = {
                'ID': original_id,
                'total_area_km2': (recoded_masked.shape[0] * recoded_masked.shape[1] * 30 * 30) / 1000000,
            }
            
            # Calculate area coverage for class 1 and class 2
            class_1_pixels = np.sum(recoded_masked == 1)
            class_2_pixels = np.sum(recoded_masked == 2)
            total_pixels = recoded_masked.shape[0] * recoded_masked.shape[1]
            
            metrics['pct_area_1'] = (class_1_pixels / total_pixels) * 100
            metrics['pct_area_2'] = (class_2_pixels / total_pixels) * 100
            metrics['area_km_1'] = (class_1_pixels * 30 * 30) / 1000000
            metrics['area_km_2'] = (class_2_pixels * 30 * 30) / 1000000
            
            # Calculate core area index for class 1 and class 2
            try:
                if class_1_pixels > 0:
                    cai_class_1 = ls.core_area_index(class_val=1, edge_depth=1, percent=True)
                    # Extract scalar value if it's a Series
                    if isinstance(cai_class_1, pd.Series):
                        metrics['cai_1'] = cai_class_1.iloc[0] if len(cai_class_1) > 0 else 0
                    else:
                        metrics['cai_1'] = cai_class_1
                else:
                    metrics['cai_1'] = 0
                    
                if class_2_pixels > 0:
                    cai_class_2 = ls.core_area_index(class_val=2, edge_depth=1, percent=True)
                    # Extract scalar value if it's a Series
                    if isinstance(cai_class_2, pd.Series):
                        metrics['cai_2'] = cai_class_2.iloc[0] if len(cai_class_2) > 0 else 0
                    else:
                        metrics['cai_2'] = cai_class_2
                else:
                    metrics['cai_2'] = 0
                    
            except Exception as e:
                print(f"Warning: Could not calculate core area index for buffer {idx}: {e}")
                metrics['cai_1'] = np.nan
                metrics['cai_2'] = np.nan
            
            buffer_metrics.append(metrics)
            
            if (idx + 1) % 10 == 0:
                print(f"Processed {idx + 1}/{len(buffered_points_raster_crs)} buffers")
                
        except Exception as e:
            print(f"Error processing buffer {idx}: {e}")
            # Add empty metrics for failed buffers
            buffer_metrics.append({
                'ID': flood_points.iloc[idx]['ID'],
                'total_area_km2': np.nan,
                'pct_area_1': np.nan,
                'pct_area_2': np.nan,
                'area_km_1': np.nan,
                'area_km_2': np.nan,
                'cai_1': np.nan,
                'cai_2': np.nan
            })

# Convert results to DataFrame
metrics_df_corrected = pd.DataFrame(buffer_metrics)
print(f"\nCompleted processing {len(buffer_metrics)} buffers")
print(f"Column names: {metrics_df_corrected.columns.tolist()}")
print("\nFirst few rows:")
print(metrics_df_corrected.head())

# Save to CSV with correct ID column
metrics_df_corrected.to_csv(r"C:\Users\alekb\Downloads\NLCD_IMP_Stats.csv", index=False)
print("Results saved to .csv")

Calculating landscape metrics for each buffer...
Processed 10/1219 buffers
Processed 20/1219 buffers
Processed 30/1219 buffers
Processed 40/1219 buffers
Processed 50/1219 buffers
Processed 60/1219 buffers
Processed 70/1219 buffers
Processed 80/1219 buffers
Processed 90/1219 buffers
Processed 100/1219 buffers
Processed 110/1219 buffers
Processed 120/1219 buffers
Processed 130/1219 buffers
Processed 140/1219 buffers
Processed 150/1219 buffers
Processed 160/1219 buffers
Processed 170/1219 buffers
Processed 180/1219 buffers
Processed 190/1219 buffers
Processed 200/1219 buffers
Processed 210/1219 buffers
Processed 220/1219 buffers
Processed 230/1219 buffers
Processed 240/1219 buffers
Processed 250/1219 buffers
Processed 260/1219 buffers
Processed 270/1219 buffers
Processed 280/1219 buffers
Processed 290/1219 buffers
Processed 300/1219 buffers
Processed 310/1219 buffers
Processed 320/1219 buffers
Processed 330/1219 buffers
Processed 340/1219 buffers
Processed 350/1219 buffers
Processed 360/1

In [29]:
# Multi-year NLCD extraction based on peak_date
import pylandstats as pls
from rasterio.mask import mask
import warnings
import os
from datetime import datetime
warnings.filterwarnings('ignore')

# Check if peak_date column exists
print("Checking flood_points columns:")
print(flood_points.columns.tolist())
print(f"\nFirst few peak_date values: {flood_points['peak_date'].head()}")

# Define NLCD file paths for each year (update paths as needed)
nlcd_files = {
    2016: r"C:\Users\alekb\Downloads\Annual_NLCD_ImpDsc_2016_CU_C1V1.tif",
    2017: r"C:\Users\alekb\Downloads\Annual_NLCD_ImpDsc_2017_CU_C1V1.tif", 
    2018: r"C:\Users\alekb\Downloads\Annual_NLCD_ImpDsc_2018_CU_C1V1.tif",
    2019: r"C:\Users\alekb\Downloads\Annual_NLCD_ImpDsc_2019_CU_C1V1.tif",
    2020: r"C:\Users\alekb\Downloads\Annual_NLCD_ImpDsc_2020_CU_C1V1.tif",
    2021: r"C:\Users\alekb\Downloads\Annual_NLCD_ImpDsc_2021_CU_C1V1.tif",
    2022: r"C:\Users\alekb\Downloads\Annual_NLCD_ImpDsc_2022_CU_C1V1.tif",
    2023: r"C:\Users\alekb\Downloads\Annual_NLCD_ImpDsc_2023_CU_C1V1.tif",
    2024: r"C:\Users\alekb\Downloads\Annual_NLCD_ImpDsc_2024_CU_C1V1.tif"
}

# Check which files exist
available_years = []
for year, filepath in nlcd_files.items():
    if os.path.exists(filepath):
        available_years.append(year)
        print(f"✓ Found NLCD file for {year}")
    else:
        print(f"✗ Missing NLCD file for {year}: {filepath}")

print(f"\nAvailable years: {available_years}")

# Extract year from peak_date column
def extract_year_from_date(date_str):
    """Extract year from various date formats"""
    try:
        if pd.isna(date_str):
            return None
        # Handle different date formats
        if isinstance(date_str, str):
            # Try common date formats
            for fmt in ['%Y-%m-%d', '%m/%d/%Y', '%Y']:
                try:
                    return datetime.strptime(date_str, fmt).year
                except ValueError:
                    continue
            # If no format works, try to extract just the year
            if len(date_str) >= 4 and date_str[:4].isdigit():
                return int(date_str[:4])
        elif hasattr(date_str, 'year'):  # datetime object
            return date_str.year
        else:
            return int(date_str) if str(date_str).isdigit() else None
    except:
        return None

# Add year column to flood_points
flood_points['year'] = flood_points['peak_date'].apply(extract_year_from_date)
print(f"\nExtracted years: {flood_points['year'].value_counts().sort_index()}")

# Initialize results
buffer_metrics = []

print("\nCalculating multi-year landscape metrics...")

# Ensure buffers are in the same CRS as raster (using 2016 as reference)
with rasterio.open(nlcd_files[2016]) as src:
    raster_crs = src.crs
    
if buffered_points.crs != raster_crs:
    buffered_points_raster_crs = buffered_points.to_crs(raster_crs)
else:
    buffered_points_raster_crs = buffered_points.copy()

# Process each buffer with its corresponding year's NLCD data
for idx, buffer_geom in enumerate(buffered_points_raster_crs.geometry):
    try:
        # Get the year for this flood point
        flood_year = flood_points.iloc[idx]['year']
        original_id = flood_points.iloc[idx]['ID']
        peak_date = flood_points.iloc[idx]['peak_date']
        
        # Use the closest available year if exact year not available
        if flood_year not in available_years:
            closest_year = min(available_years, key=lambda x: abs(x - flood_year)) if available_years else 2016
            print(f"Year {flood_year} not available for ID {original_id}, using {closest_year}")
            flood_year = closest_year
        
        # Open the appropriate NLCD file for this year
        nlcd_file = nlcd_files[flood_year]
        
        with rasterio.open(nlcd_file) as src:
            # Mask the raster data with the buffer geometry
            masked_data, masked_transform = mask(src, [buffer_geom], crop=True, filled=False)
            masked_array = masked_data[0]
            
            # Replace masked values
            masked_array = np.where(masked_array.mask, -1, masked_array.data)
            
            # Apply recoding
            recoded_masked = np.where((masked_array == 1) | (masked_array == 2), masked_array, 0)
            
            # Create Landscape object
            ls = pls.Landscape(recoded_masked, res=(30, 30))
            
            # Calculate metrics
            metrics = {
                'ID': original_id,
                'peak_date': peak_date,
                'nlcd_year': flood_year,
                'total_area_km2': (recoded_masked.shape[0] * recoded_masked.shape[1] * 30 * 30) / 1000000,
            }
            
            # Calculate area coverage
            class_1_pixels = np.sum(recoded_masked == 1)
            class_2_pixels = np.sum(recoded_masked == 2)
            total_pixels = recoded_masked.shape[0] * recoded_masked.shape[1]
            
            metrics['pct_area_1'] = (class_1_pixels / total_pixels) * 100
            metrics['pct_area_2'] = (class_2_pixels / total_pixels) * 100
            metrics['area_km_1'] = (class_1_pixels * 30 * 30) / 1000000
            metrics['area_km_2'] = (class_2_pixels * 30 * 30) / 1000000
            
            # Calculate core area index
            try:
                if class_1_pixels > 0:
                    cai_class_1 = ls.core_area_index(class_val=1, edge_depth=1, percent=True)
                    metrics['cai_1'] = cai_class_1.iloc[0] if isinstance(cai_class_1, pd.Series) else cai_class_1
                else:
                    metrics['cai_1'] = 0
                    
                if class_2_pixels > 0:
                    cai_class_2 = ls.core_area_index(class_val=2, edge_depth=1, percent=True)
                    metrics['cai_2'] = cai_class_2.iloc[0] if isinstance(cai_class_2, pd.Series) else cai_class_2
                else:
                    metrics['cai_2'] = 0
            except Exception as e:
                print(f"Warning: Could not calculate CAI for ID {original_id}: {e}")
                metrics['cai_1'] = np.nan
                metrics['cai_2'] = np.nan
            
            buffer_metrics.append(metrics)
            
            if (idx + 1) % 10 == 0:
                print(f"Processed {idx + 1}/{len(buffered_points_raster_crs)} buffers")
                
    except Exception as e:
        print(f"Error processing buffer {idx} (ID: {original_id}): {e}")
        buffer_metrics.append({
            'ID': original_id,
            'peak_date': peak_date,
            'nlcd_year': np.nan,
            'total_area_km2': np.nan,
            'pct_area_1': np.nan,
            'pct_area_2': np.nan,
            'area_km_1': np.nan,
            'area_km_2': np.nan,
            'cai_1': np.nan,
            'cai_2': np.nan
        })

# Create final DataFrame
multi_year_metrics_df = pd.DataFrame(buffer_metrics)
print(f"\nCompleted processing {len(buffer_metrics)} buffers")
print(f"Years used: {multi_year_metrics_df['nlcd_year'].value_counts().sort_index()}")
print("\nFirst few rows:")
print(multi_year_metrics_df.head())

# Save results
multi_year_metrics_df.to_csv(r"C:\Users\alekb\Downloads\NLCD_MultiYear_Stats.csv", index=False)
print("Multi-year results saved to NLCD_MultiYear_Stats.csv")

Checking flood_points columns:
['latitude_d', 'longitude_', 'latitude', 'longitude', 'eventName', 'hwmTypeNam', 'hwmQuality', 'verticalDa', 'verticalMe', 'approvalMe', 'markerName', 'horizontal', 'horizont_1', 'flagMember', 'surveyMemb', 'site_no', 'siteDescri', 'sitePriori', 'networkNam', 'stateName', 'countyName', 'siteZone', 'sitePermHo', 'site_latit', 'site_longi', 'hwm_id', 'waterbody', 'site_id', 'event_id', 'hwm_type_i', 'hwm_qualit', 'hwm_locati', 'survey_dat', 'elev_ft', 'vdatum_id', 'vcollect_m', 'bank', 'approval_i', 'marker_id', 'height_abo', 'hcollect_m', 'peak_summa', 'hwm_notes', 'hwm_enviro', 'flag_date', 'stillwater', 'hdatum_id', 'flag_membe', 'survey_mem', 'uncertaint', 'hwm_uncert', 'hwm_label', 'last_updat', 'last_upd_1', 'horizont_2', 'vertical_c', 'horizont_3', 'hwm_types', 'hwm_qual_1', 'event', 'peak_sum_1', 'survey_m_1', 'marker', 'approval', 'files', 'site', 'vertical_d', 'flag_mem_1', 'Links', 'p_latitude', 'p_longitud', 'p_vdatum', 'p_member_n', 'p_site_id'